# Importing Libraries and Dataset

In [103]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from imblearn.over_sampling import SMOTE
from collections import Counter

from sklearn.model_selection import train_test_split

In [104]:
pd.options.display.max_rows = 100
pd.options.display.max_columns = 200

# Read Merged Dataset

In [106]:
df = pd.read_csv('data/india_fraud.csv')
df

,CustomerID,ReportedFraud,DateOfIncident,TypeOfIncident,TypeOfCollission,SeverityOfIncident,AuthoritiesContacted,IncidentState,IncidentCity,IncidentAddress,IncidentTime,NumberOfVehicles,PropertyDamage,BodilyInjuries,Witnesses,PoliceReport,AmountOfTotalClaim,AmountOfInjuryClaim,AmountOfPropertyClaim,AmountOfVehicleDamage,InsuredAge,InsuredZipCode,InsuredGender,InsuredEducationLevel,InsuredOccupation,InsuredHobbies,CapitalGains,CapitalLoss,Country,InsurancePolicyNumber,CustomerLoyaltyPeriod,DateOfPolicyCoverage,InsurancePolicyState,Policy_CombinedSingleLimit,Policy_Deductible,PolicyAnnualPremium,UmbrellaLimit,InsuredRelationship,VehicleID,VehicleMake,VehicleModel,VehicleYOM
0,Cust20065,N,2015-01-09,Multi-vehicle Collision,Rear Collision,Total Loss,Other,State9,City4,Location 1404,19,3,?,1,1,?,43973,9396,4698,29879,58,471784,MALE,JD,sales,video-games,0,-42700,India,129186,432,1998-11-14,State2,250/500,2000,1142.87,0,own-child,Vehicle30662,Mercedes,E400,2005
1,Cust37589,N,2015-02-20,Single Vehicle Collision,Side Collision,Minor Damage,Ambulance,State4,City3,Location 1675,14,1,?,1,1,YES,62310,6539,6539,49232,34,431937,FEMALE,High School,transport-moving,polo,51300,0,India,146710,128,2000-08-19,State2,100/300,1000,1274.38,2857344,own-child,Vehicle17183,Toyota,Highlander,2010
2,Cust24312,N,2015-01-14,Multi-vehicle Collision,Side Collision,Major Damage,Ambulance,State5,City2,Location 1546,21,3,?,0,0,?,42824,6069,6069,30686,52,479320,FEMALE,PhD,machine-op-inspct,exercise,0,0,India,133433,346,2000-06-23,State1,500/1000,745,1269.93,0,other-relative,Vehicle33982,Volkswagen,Passat,2002
3,Cust5493,Y,2015-01-07,Multi-vehicle Collision,Side Collision,Major Damage,Ambulance,State7,City4,Location 1413,12,3,YES,2,0,?,45672,253,5741,39678,25,607763,FEMALE,College,exec-managerial,exercise,47400,-56100,India,114614,42,1992-01-15,State2,100/300,986,1218.60,0,other-relative,Vehicle12299,Toyota,Highlander,2011
4,Cust7704,Y,2015-02-26,Multi-vehicle Collision,Side Collision,Major Damage,Other,State4,City6,Location 1367,3,3,NO,1,2,NO,81472,7407,14813,59252,27,441783,MALE,Masters,sales,chess,0,0,India,116825,109,2001-09-23,State3,250/500,576,1431.02,4235779,unmarried,Vehicle30447,Ford,Wrangler,2010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28831,Cust9773,N,2015-02-13,Single Vehicle Collision,Side Collision,Minor Damage,Fire,State7,City3,Location 1324,16,1,NO,1,2,?,50515,5543,5543,39429,39,454139,MALE,JD,farming-fishing,basketball,0,0,India,118894,251,2007-02-07,State2,250/500,1000,1108.55,739864,unmarried,Vehicle24503,Nissan,RSX,2002
28832,Cust38439,N,2015-01-19,Multi-vehicle Collision,Side Collision,Total Loss,Police,State5,City3,Location 1880,19,3,YES,2,0,?,94920,8678,8738,77504,21,463809,MALE,Associate,prof-specialty,golf,20000,-75000,India,147560,14,2006-10-18,State2,100/300,1008,1551.84,41592,own-child,Vehicle9733,Accura,RSX,2014
28833,Cust38444,N,2015-01-21,Multi-vehicle Collision,Side Collision,Total Loss,Police,State5,City7,Location 1632,16,3,YES,2,0,NO,87893,13224,7990,66679,35,444155,MALE,Associate,prof-specialty,golf,0,-75000,India,147565,153,2011-11-23,State1,100/300,660,1119.32,0,not-in-family,Vehicle1823,Saab,RSX,2013
28834,Cust26466,N,2015-01-02,Parked Car,?,Trivial Damage,None,State4,City7,Location 1364,12,1,?,1,1,?,7284,728,1457,5099,40,603123,FEMALE,Masters,exec-managerial,reading,0,-63900,India,135587,241,1997-08-04,State3,250/1000,1674,1229.88,4674160,wife,Vehicle9113,Accura,RSX,2004


# Handling Missing Values

#### Looking for NA values

In [107]:
df.isna().sum() 

CustomerID                     0
ReportedFraud                  0
DateOfIncident                 0
TypeOfIncident                 0
TypeOfCollission               0
SeverityOfIncident             0
AuthoritiesContacted           0
IncidentState                  0
IncidentCity                   0
IncidentAddress                0
IncidentTime                   0
NumberOfVehicles               0
PropertyDamage                 0
BodilyInjuries                 0
Witnesses                      0
PoliceReport                   0
AmountOfTotalClaim             0
AmountOfInjuryClaim            0
AmountOfPropertyClaim          0
AmountOfVehicleDamage          0
InsuredAge                     0
InsuredZipCode                 0
InsuredGender                 30
InsuredEducationLevel          0
InsuredOccupation              0
InsuredHobbies                 0
CapitalGains                   0
CapitalLoss                    0
Country                        2
InsurancePolicyNumber          0
CustomerLo

#### Deep-diving into each column

In [108]:
for col in df:
    print('====================')
    print(col)
    print(df[col].unique())
    print(df[col].value_counts())

# Findings: 
# Categorical columns: 
# Columns 'TypeOfCollision', 'PropertyDamage', 'PoliceReport' have a '?' category.
# 'VehicleMake' column have a '???' category.
# Solution: As the '?'/'???' category take up a significant proportion of the dataset, we will be leaving the category as it is and 
# consider it as a 'missing' category. We will rename it as 'Missing' to be neater.

# Numerical columns:
# 'Witnesses' column have a 'MISSINGVALUE' category.
# 'AmountOfTotalClaim' column have a 'MISSEDDATA' category.
# Solution for 'Witnesses' column: We will leave the category as it is and look at this column as a catgorical column.
# Solution for 'AmountOfTotalClaim' column: Removing, using column that has high correlation

CustomerID
['Cust20065' 'Cust37589' 'Cust24312' ... 'Cust38444' 'Cust26466'
 'Cust29555']
Cust20065    1
Cust24317    1
Cust33972    1
Cust34480    1
Cust36994    1
            ..
Cust10466    1
Cust20090    1
Cust21677    1
Cust24266    1
Cust29555    1
Name: CustomerID, Length: 28836, dtype: int64
ReportedFraud
['N' 'Y']
N    21051
Y     7785
Name: ReportedFraud, dtype: int64
DateOfIncident
['2015-01-09' '2015-02-20' '2015-01-14' '2015-01-07' '2015-02-26'
 '2015-02-06' '2015-02-02' '2015-01-26' '2015-02-08' '2015-01-15'
 '2015-02-14' '2015-02-04' '2015-01-16' '2015-02-27' '2015-01-19'
 '2015-02-16' '2015-01-11' '2015-01-28' '2015-02-19' '2015-02-05'
 '2015-02-21' '2015-01-13' '2015-01-06' '2015-01-17' '2015-01-08'
 '2015-02-07' '2015-01-12' '2015-02-15' '2015-01-18' '2015-01-05'
 '2015-02-10' '2015-01-25' '2015-02-11' '2015-02-28' '2015-01-10'
 '2015-03-04' '2015-01-22' '2015-02-17' '2015-01-21' '2015-02-25'
 '2015-01-27' '2015-01-04' '2015-02-18' '2015-01-29' '2015-02-13'
 '2015-01-

#### Dropping Columns/Rows

##### Drop Categorical Columns

In [109]:
# Drop 'Country' column as India is the only country in the entire dataset and there are only 2 rows with missing country
# Drop 'InsuredGender' due to correlation with other categorical features
# Drop 'InsuredHobbies' due to possible spurious correlation with other features
df = df.drop(columns=['Country','InsuredGender','InsuredHobbies'])

#### Drop Numerical Columns

In [110]:
# **ONLY KEEP AmountOfVehicleDamage** : 
# High Corr between AmountOfTotalClaim, AmountOfInjuryClaim, AmountOfPropertyClaim and AmountOfVehicleDamage. 
# Think AmountOfTotalClaim would be most important, but there are alot of NAs. Hence, taking the value with highest correlation with AmountOfTotalClaim, which is AmountOfVehicleDamage.
# Remove InsuredAge as it has high correlation with InsurancePolicyState
df = df.drop(columns=['AmountOfTotalClaim','AmountOfInjuryClaim','AmountOfPropertyClaim','InsuredAge'])

##### Drop Other Redundant Columns

In [111]:
# 'CustomerID', 'VehicleID', 'InsurancePolicyNumber' are just IDs and are not useful.
# 'IncidentState' and 'IncidentCity' are captured by 'IncidentAddress'.
# 'InsuredZipCode' captured by 'InsurancePolicyState'
# 'DateOfPolicyCoverage' is aligned with 'CustomerLoyaltyPeriod' 
df.drop(columns=['CustomerID', 'VehicleID', 'InsurancePolicyNumber', 'InsuredZipCode', 'IncidentState','IncidentCity','DateOfPolicyCoverage'], inplace=True)

#### Replacing column values for certain columns

In [112]:
# Replacing '?'/'???' value with 'Missing' for 'TypeOfCollision', 'PropertyDamage', 'PoliceReport', 'Witnesses' and
# 'VehicleMake' columns as explained above.
df['TypeOfCollission'] = df['TypeOfCollission'].replace('?','Missing')
df['PropertyDamage'] = df['PropertyDamage'].replace('?','Missing')
df['PoliceReport'] = df['PoliceReport'].replace('?','Missing')
df['VehicleMake'] = df['VehicleMake'].replace('???','Missing')
df['Witnesses'] = df['Witnesses'].replace('MISSINGVALUE','-1')

# Changing 'Witnesses' to integer
df['Witnesses'] = df['Witnesses'].astype(int)

In [113]:
# Changing PropertyDamage & PoliceReport to numerical variables (-1 for missing, add '{Feature}_Missing' column), since they are naturally binary variables
def change_binary(x):
  if x == 'YES':
    return 1
  elif x == 'NO':
    return 0
  elif x == 'Missing':
    return -1

df['PropertyDamage'] = df['PropertyDamage'].apply(lambda x: change_binary(x))
df['PropertyDamage_Missing'] = df['PropertyDamage'].apply(lambda x: 1 if x == -1 else 0)
df['PoliceReport'] = df['PoliceReport'].apply(lambda x: change_binary(x))
df['PoliceReport_Missing'] = df['PoliceReport'].apply(lambda x: 1 if x == -1 else 0)

In [114]:
# Adding feature **UmbrellaLimit_Missing** : since most of them are 0 not because the insurance value is 0, but because they do not have insurance for 3rd party
df['UmbrellaLimit_Missing'] = df['UmbrellaLimit'].apply(lambda x: 1 if x == 0 else 0)

In [115]:
# Replacing 'Y' and 'N' with 1 and 0 for target column, 'ReportedFraud' for prediction
df['ReportedFraud'] = [1 if x == 'Y' else 0 for x in df['ReportedFraud']]

# Extracting information for date columns

In [116]:
# Extract month and day from 'DateOfIncident' column. Year not extracted as the all incidents happened in 2015.
df['MonthOfIncident'] = pd.to_datetime(df['DateOfIncident']).dt.month
df['DayOfIncident'] = pd.to_datetime(df['DateOfIncident']).dt.day

df = df.drop(columns=['DateOfIncident'])

In [117]:
df

,ReportedFraud,TypeOfIncident,TypeOfCollission,SeverityOfIncident,AuthoritiesContacted,IncidentAddress,IncidentTime,NumberOfVehicles,PropertyDamage,BodilyInjuries,Witnesses,PoliceReport,AmountOfVehicleDamage,InsuredAge,InsuredEducationLevel,InsuredOccupation,CapitalGains,CapitalLoss,CustomerLoyaltyPeriod,InsurancePolicyState,Policy_CombinedSingleLimit,Policy_Deductible,PolicyAnnualPremium,UmbrellaLimit,InsuredRelationship,VehicleMake,VehicleModel,VehicleYOM,PropertyDamage_Missing,PoliceReport_Missing,UmbrellaLimit_Missing,MonthOfIncident,DayOfIncident
0,0,Multi-vehicle Collision,Rear Collision,Total Loss,Other,Location 1404,19,3,-1,1,1,-1,29879,58,JD,sales,0,-42700,432,State2,250/500,2000,1142.87,0,own-child,Mercedes,E400,2005,1,1,1,1,9
1,0,Single Vehicle Collision,Side Collision,Minor Damage,Ambulance,Location 1675,14,1,-1,1,1,1,49232,34,High School,transport-moving,51300,0,128,State2,100/300,1000,1274.38,2857344,own-child,Toyota,Highlander,2010,1,0,0,2,20
2,0,Multi-vehicle Collision,Side Collision,Major Damage,Ambulance,Location 1546,21,3,-1,0,0,-1,30686,52,PhD,machine-op-inspct,0,0,346,State1,500/1000,745,1269.93,0,other-relative,Volkswagen,Passat,2002,1,1,1,1,14
3,1,Multi-vehicle Collision,Side Collision,Major Damage,Ambulance,Location 1413,12,3,1,2,0,-1,39678,25,College,exec-managerial,47400,-56100,42,State2,100/300,986,1218.60,0,other-relative,Toyota,Highlander,2011,0,1,1,1,7
4,1,Multi-vehicle Collision,Side Collision,Major Damage,Other,Location 1367,3,3,0,1,2,0,59252,27,Masters,sales,0,0,109,State3,250/500,576,1431.02,4235779,unmarried,Ford,Wrangler,2010,0,0,0,2,26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28831,0,Single Vehicle Collision,Side Collision,Minor Damage,Fire,Location 1324,16,1,0,1,2,-1,39429,39,JD,farming-fishing,0,0,251,State2,250/500,1000,1108.55,739864,unmarried,Nissan,RSX,2002,0,1,0,2,13
28832,0,Multi-vehicle Collision,Side Collision,Total Loss,Police,Location 1880,19,3,1,2,0,-1,77504,21,Associate,prof-specialty,20000,-75000,14,State2,100/300,1008,1551.84,41592,own-child,Accura,RSX,2014,0,1,0,1,19
28833,0,Multi-vehicle Collision,Side Collision,Total Loss,Police,Location 1632,16,3,1,2,0,0,66679,35,Associate,prof-specialty,0,-75000,153,State1,100/300,660,1119.32,0,not-in-family,Saab,RSX,2013,0,0,1,1,21
28834,0,Parked Car,Missing,Trivial Damage,None,Location 1364,12,1,-1,1,1,-1,5099,40,Masters,exec-managerial,0,-63900,241,State3,250/1000,1674,1229.88,4674160,wife,Accura,RSX,2004,1,1,0,1,2


In [118]:
df.to_csv('data/india_fraud_cleaned.csv', index=False)

# Train Test Split

In [119]:
# split train test val in 80/10/10
train, test = train_test_split(df, stratify=df['ReportedFraud'], test_size=0.1)
train, val = train_test_split(train, stratify=df.loc[train.index, "ReportedFraud"], test_size=1/9)

# Create Suspicious Location Variable for train, test, val set

Divide into train test split, get **suspicious locations**, change it to a binary value instead (Suspicious Location or Not, -1 would be not found (in val/test set)

In [120]:
import plotly.express as px
fig = px.histogram(train[['IncidentAddress','ReportedFraud']].value_counts().reset_index().rename(columns={0:'cases'}), x='IncidentAddress',y='cases',color='ReportedFraud',barmode='group',title=f'Number of Reported Fraud Cases By IncidentAddress')
fig.show()

In [121]:
location_breakdown = pd.pivot_table(train[['IncidentAddress','ReportedFraud']].value_counts().reset_index().rename(columns={0:'cases'}).sort_values(by='IncidentAddress'), index='IncidentAddress', columns=['ReportedFraud'], values='cases', aggfunc=np.sum).fillna(0).reset_index()
location_breakdown['Suspicious_Location'] = (location_breakdown[0] < location_breakdown[1]) * 1
location_breakdown

ReportedFraud,IncidentAddress,0,1,Suspicious_Location
0,Location 1001,1.0,13.0,1
1,Location 1002,12.0,2.0,0
2,Location 1003,7.0,1.0,0
3,Location 1004,38.0,3.0,0
4,Location 1005,20.0,0.0,0
...,...,...,...,...
995,Location 2096,22.0,2.0,0
996,Location 2097,42.0,1.0,0
997,Location 2098,30.0,3.0,0
998,Location 2099,30.0,2.0,0


In [122]:
location_breakdown[location_breakdown['Suspicious_Location'] == 1]

ReportedFraud,IncidentAddress,0,1,Suspicious_Location
0,Location 1001,1.0,13.0,1
7,Location 1008,1.0,17.0,1
13,Location 1014,1.0,30.0,1
17,Location 1018,1.0,10.0,1
18,Location 1019,0.0,18.0,1
...,...,...,...,...
984,Location 2085,0.0,14.0,1
985,Location 2086,0.0,13.0,1
987,Location 2088,5.0,15.0,1
992,Location 2093,0.0,10.0,1


In [123]:
suspicious_locations = list(location_breakdown[location_breakdown['Suspicious_Location'] == 1]['IncidentAddress'])

In [124]:
len(suspicious_locations)

247

In [125]:
all_locations_train = list(location_breakdown['IncidentAddress'])

In [126]:
def in_suspicious_location(x):
  if x in all_locations_train:
    if x in suspicious_locations:
      return 1
    else:
      return 0
  else:
    return -1  # mainly for val and test, when location is not in training set.


train['SuspiciousLocation'] = train['IncidentAddress'].apply(lambda x: in_suspicious_location(x))
train = train.drop(columns=['IncidentAddress'])

val['SuspiciousLocation'] = val['IncidentAddress'].apply(lambda x: in_suspicious_location(x))
val = val.drop(columns=['IncidentAddress'])

test['SuspiciousLocation'] = test['IncidentAddress'].apply(lambda x: in_suspicious_location(x))
test = test.drop(columns=['IncidentAddress'])

# One Hot Encoding on Categorical Variables

In [127]:
cat_cols = ['TypeOfIncident', 'TypeOfCollission', 'SeverityOfIncident', 'AuthoritiesContacted','InsuredEducationLevel',
       'InsuredOccupation', 'InsurancePolicyState', 'Policy_CombinedSingleLimit', 'InsuredRelationship',
       'VehicleMake', 'VehicleModel']
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numeric_cols = train.select_dtypes(include=numerics).columns.tolist()

In [128]:
# OHE for train set
dummy_cols = pd.get_dummies(train[cat_cols])
dummy_cols
train_ohe = pd.concat([train[numeric_cols], dummy_cols], axis = 1)
train_ohe.shape

(23068, 132)

In [129]:
# OHE for test set
dummy_cols = pd.get_dummies(test[cat_cols])
dummy_cols
test_ohe = pd.concat([test[numeric_cols], dummy_cols], axis = 1)
test_ohe.shape

(2884, 132)

In [130]:
# OHE for val set
dummy_cols = pd.get_dummies(val[cat_cols])
dummy_cols
val_ohe = pd.concat([val[numeric_cols], dummy_cols], axis = 1)
val_ohe.shape

(2884, 132)

In [131]:
# Split IV and DV
X_train = train_ohe.iloc[:,1:]
y_train = train_ohe['ReportedFraud'] 

X_val = val_ohe.iloc[:,1:]
y_val = val_ohe['ReportedFraud']

X_test = test_ohe.iloc[:,1:]
y_test = test_ohe['ReportedFraud']

## Dealing with Class Imbalance using SMOTE

In [132]:
smote = SMOTE()

# fit predictor and target variable
X_smote, y_smote = smote.fit_resample(X_train, y_train)
print('Original dataset shape', Counter(y_train))
print('Resample dataset shape', Counter(y_smote))

Original dataset shape Counter({0: 16841, 1: 6227})
Resample dataset shape Counter({0: 16841, 1: 16841})


In [133]:
X_smote.to_csv('data/X_smote.csv', index=False)
y_smote.to_csv('data/y_smote.csv', index=False)

X_test.to_csv('data/X_test.csv', index=False)
y_test.to_csv('data/y_test.csv', index=False)

X_val.to_csv('data/X_val.csv', index=False)
y_val.to_csv('data/y_val.csv', index=False)